# 特定の日時のリストを取得する 

In [1]:
%cd ..

E:\システムトレード入門\trade_system_git_workspace


In [2]:
import jpholiday
import workdays
from pytz import timezone, utc
import datetime

In [3]:
import numpy as np
import pandas as pd

In [4]:
def check_jst_datetimes_to_naive(*arg_datetimes):
    """
    ＊*今のところ，ローカルが東京でないnaiveなdatetimeはそのまま通してしまう
    引数のタイムゾーンが同じかどうかチェックし，存在するなら日本であるかチェック
    awareな場合は，naiveに変更
    """
    jst_timezone = timezone("Asia/Tokyo")
    tz_info_set = set([one_datetime.tzinfo for one_datetime in arg_datetimes])
    if len(tz_info_set) > 1:
        raise Exception("timezones are different")
        
    datetimes_tzinfo = list(tz_info_set)[0]
    
    if datetimes_tzinfo is not None:  # 長さが1のはず
        if timezone(str(datetimes_tzinfo)) != jst_timezone:
            raise Exception("timezones must be Asia/Tokyo")
        # naiveなdatetimeに変更
        arg_datetimes = [one_datetime.replace(tzinfo=None) for one_datetime in arg_datetimes]
        
    return tuple(arg_datetimes)

## 営業日・休日を取得する 

### 祝日のndarrayを取得する

以下が素直な実装だが，遅い．そこで一度しか呼ばないようにすべき

In [5]:
def get_holiday_jp(start_datetime, end_datetime, with_name=False, to_date=True):
    """
    期間を指定して祝日を取得
    
    start_datetime: datetime.datetime
        開始時刻のjstのタイムゾーンが指定されたawareなdatetimeか，naiveなdatetime(ローカルなタイムゾーンとして日本を仮定)
    end_datetime: datetime.datetime
        終了時刻のjstのタイムゾーンが指定されたawareなdatetimeか，naiveなdatetime(ローカルなタイムゾーンとして日本を仮定)
    eith_name: bool
        休日の名前を出力するかどうか
    to_date: bool
        出力をdatetime.datetimeにするかdatetime.dateにするか
    """
    start_datetime, end_datetime = check_jst_datetimes_to_naive(start_datetime, end_datetime)
    
    if to_date:
        start_datetime = start_datetime.date()
        end_datetime = end_datetime.date()
    
    holydays_array = np.array(jpholiday.between(start_datetime, end_datetime))
    
    if not with_name:
        return holydays_array[:,0].copy()
    
    return holydays_array

In [6]:
start_datetime = datetime.datetime(2019, 1, 1, 0, 0, 0)
end_datetime = datetime.datetime(2020, 12, 31, 0, 0, 0)

#jst_timezone = timezone("Asia/Tokyo")
#start_datetime = jst_timezone.localize(datetime.datetime(2019, 1, 1, 0, 0, 0))
#end_datetime = jst_timezone.localize(datetime.datetime(2020, 12, 31, 0, 0, 0))

holydays = get_holiday_jp(start_datetime, end_datetime, with_name=False, to_date=True)

In [7]:
holydays

array([datetime.date(2019, 1, 1), datetime.date(2019, 1, 14),
       datetime.date(2019, 2, 11), datetime.date(2019, 3, 21),
       datetime.date(2019, 4, 29), datetime.date(2019, 4, 30),
       datetime.date(2019, 5, 1), datetime.date(2019, 5, 2),
       datetime.date(2019, 5, 3), datetime.date(2019, 5, 4),
       datetime.date(2019, 5, 5), datetime.date(2019, 5, 6),
       datetime.date(2019, 7, 15), datetime.date(2019, 8, 11),
       datetime.date(2019, 8, 12), datetime.date(2019, 9, 16),
       datetime.date(2019, 9, 23), datetime.date(2019, 10, 14),
       datetime.date(2019, 10, 22), datetime.date(2019, 11, 3),
       datetime.date(2019, 11, 4), datetime.date(2019, 11, 23),
       datetime.date(2020, 1, 1), datetime.date(2020, 1, 13),
       datetime.date(2020, 2, 11), datetime.date(2020, 2, 23),
       datetime.date(2020, 2, 24), datetime.date(2020, 3, 20),
       datetime.date(2020, 4, 29), datetime.date(2020, 5, 3),
       datetime.date(2020, 5, 4), datetime.date(2020, 5, 5),


### 営業日を出力する 

たくさんのライブラリを利用するのは面倒なので，自分で作成する．以下が素直な実装だが，遅い．

In [8]:
def get_workdays_jp(start_datetime, end_datetime, to_date=True, end_include=False):
    """
    営業日を取得
    
    start_datetime: datetime.datetime
        開始時刻のjstのタイムゾーンが指定されたawareなdatetimeか，naiveなdatetime(ローカルなタイムゾーンとして日本を仮定)
    end_datetime: datetime.datetime
        終了時刻のjstのタイムゾーンが指定されたawareなdatetimeか，naiveなdatetime(ローカルなタイムゾーンとして日本を仮定)
    to_date: bool
        出力をdatetime.datetimeにするかdatetime.dateにするか
    end_include: bool
        最終日も含めて出力するか
    """
    start_datetime, end_datetime = check_jst_datetimes_to_naive(start_datetime, end_datetime)
    
    def get_workdays_gen_jp(start_datetime, end_datetime):
        start_date = start_datetime.date()
        end_date = end_datetime.date()
        if end_include:
            time_delta_days = (end_date - start_date).days + 1  # 最終日も含める
        else:
            time_delta_days = (end_date - start_date).days  # 最終日は含めない
            
        for i in range(time_delta_days):
            out_datetime = start_datetime + datetime.timedelta(days=i)
            if out_datetime.weekday() not in (5,6):  # 土日ではない
                if not jpholiday.is_holiday(out_datetime):  # 休日ではない
                    if to_date:
                        out_datetime = out_datetime.date()
                    yield out_datetime
        
    # workdayのリストを取得
    workday_array = np.array([workday for workday in get_workdays_gen_jp(start_datetime, end_datetime)])
    return workday_array

In [11]:
#start_datetime = datetime.datetime(2019, 1, 1, 0, 0, 0)
#end_datetime = datetime.datetime(2019, 12, 31, 0, 0, 0)

jst_timezone = timezone("Asia/Tokyo")
start_datetime = jst_timezone.localize(datetime.datetime(2020, 1, 1, 0, 0, 0))
end_datetime = jst_timezone.localize(datetime.datetime(2020, 12, 31, 0, 0, 0))

workdays1 = get_workdays_jp(start_datetime, end_datetime, end_include=True)

In [12]:
workdays1

array([datetime.date(2020, 1, 2), datetime.date(2020, 1, 3),
       datetime.date(2020, 1, 6), datetime.date(2020, 1, 7),
       datetime.date(2020, 1, 8), datetime.date(2020, 1, 9),
       datetime.date(2020, 1, 10), datetime.date(2020, 1, 14),
       datetime.date(2020, 1, 15), datetime.date(2020, 1, 16),
       datetime.date(2020, 1, 17), datetime.date(2020, 1, 20),
       datetime.date(2020, 1, 21), datetime.date(2020, 1, 22),
       datetime.date(2020, 1, 23), datetime.date(2020, 1, 24),
       datetime.date(2020, 1, 27), datetime.date(2020, 1, 28),
       datetime.date(2020, 1, 29), datetime.date(2020, 1, 30),
       datetime.date(2020, 1, 31), datetime.date(2020, 2, 3),
       datetime.date(2020, 2, 4), datetime.date(2020, 2, 5),
       datetime.date(2020, 2, 6), datetime.date(2020, 2, 7),
       datetime.date(2020, 2, 10), datetime.date(2020, 2, 12),
       datetime.date(2020, 2, 13), datetime.date(2020, 2, 14),
       datetime.date(2020, 2, 17), datetime.date(2020, 2, 18),
   

内部で外部の関数等を利用するので，jitコンパイルはできない．そこで，あらかじめ数年分のholidayを得て置き，それと照らし合わせることにする．

In [13]:
holiday_start_year = 2015
holiday_end_year = 2020

holidays_date_array = get_holiday_jp(start_datetime=datetime.datetime(holiday_start_year,1,1,0,0,0),
                                end_datetime=datetime.datetime(holiday_end_year,12,31,23,59,59),
                                with_name=False,
                               )
#print(holidays_date_array[:10])

holidays_datetimeindex = pd.DatetimeIndex(holidays_date_array)
print(holidays_datetimeindex)

DatetimeIndex(['2015-01-01', '2015-01-12', '2015-02-11', '2015-03-21',
               '2015-04-29', '2015-05-03', '2015-05-04', '2015-05-05',
               '2015-05-06', '2015-07-20',
               ...
               '2020-05-04', '2020-05-05', '2020-05-06', '2020-07-23',
               '2020-07-24', '2020-08-10', '2020-09-21', '2020-09-22',
               '2020-11-03', '2020-11-23'],
              dtype='datetime64[ns]', length=111, freq=None)


In [14]:
def convert_datetime2date(one_datetime):
    return one_datetime.date()

# datetieをdateに変換するuniversal function
convert_datetime2date_ufunc = np.frompyfunc(convert_datetime2date, 1, 1)


def get_workdays_jp_v2(start_datetime, end_datetime, to_date=True, end_include=False):
    """
    営業日を取得
    
    start_datetime: datetime.datetime
        開始時刻のjstのタイムゾーンが指定されたawareなdatetimeか，naiveなdatetime(ローカルなタイムゾーンとして日本を仮定)
    end_datetime: datetime.datetime
        終了時刻のjstのタイムゾーンが指定されたawareなdatetimeか，naiveなdatetime(ローカルなタイムゾーンとして日本を仮定)
    to_date: bool
        出力をdatetime.datetimeにするかdatetime.dateにするか
    end_include: bool
        最終日も含めて出力するか
    """
    start_datetime, end_datetime = check_jst_datetimes_to_naive(start_datetime, end_datetime)
    
    # 期間中のholidayを取得
    holidays_in_span_datetimeindex = holidays_datetimeindex[(start_datetime<=holidays_datetimeindex)&(holidays_datetimeindex<end_datetime)].copy()
    holidays_in_span_array = holidays_in_span_datetimeindex.values

    # 期間中のdatetimeのarrayを取得
    if end_include:
        days_siries = pd.date_range(start=start_datetime, end=end_datetime, freq="D")  # 最終日も含める
    else:
        days_siries = pd.date_range(start=start_datetime, end=end_datetime-datetime.timedelta(days=1), freq="D")  # 最終日は含めない
    
    days_datetimeindex = pd.DatetimeIndex(days_siries)
    
    # 休日に含まれないもの，さらに土日に含まれないもののboolインデックスを取得
    holiday_bool_array = days_datetimeindex.isin(holidays_in_span_array)  # 休日
    sun_or_satur_bool_array = (days_datetimeindex.weekday==5) | (days_datetimeindex.weekday==6)  # 土曜日or日曜日
    
    workdays_bool_array = (~holiday_bool_array)&(~sun_or_satur_bool_array)  # 休日でなく土日でない
    
    workdays_datetimeindex = days_datetimeindex[workdays_bool_array].copy()
    workdays_array = workdays_datetimeindex.to_pydatetime()
    if to_date:
        workdays_array = convert_datetime2date_ufunc(workdays_array)
        
    return workdays_array

In [19]:
#start_datetime = datetime.datetime(2019, 1, 1, 0, 0, 0)
#end_datetime = datetime.datetime(2019, 12, 31, 0, 0, 0)

jst_timezone = timezone("Asia/Tokyo")
start_datetime = jst_timezone.localize(datetime.datetime(2020, 1, 1, 0, 0, 0))
end_datetime = jst_timezone.localize(datetime.datetime(2020, 12, 31, 0, 0, 0))

workdays2 = get_workdays_jp_v2(start_datetime, end_datetime, end_include=True)

In [20]:
workdays2

array([datetime.date(2020, 1, 2), datetime.date(2020, 1, 3),
       datetime.date(2020, 1, 6), datetime.date(2020, 1, 7),
       datetime.date(2020, 1, 8), datetime.date(2020, 1, 9),
       datetime.date(2020, 1, 10), datetime.date(2020, 1, 13),
       datetime.date(2020, 1, 14), datetime.date(2020, 1, 15),
       datetime.date(2020, 1, 16), datetime.date(2020, 1, 17),
       datetime.date(2020, 1, 20), datetime.date(2020, 1, 21),
       datetime.date(2020, 1, 22), datetime.date(2020, 1, 23),
       datetime.date(2020, 1, 24), datetime.date(2020, 1, 27),
       datetime.date(2020, 1, 28), datetime.date(2020, 1, 29),
       datetime.date(2020, 1, 30), datetime.date(2020, 1, 31),
       datetime.date(2020, 2, 3), datetime.date(2020, 2, 4),
       datetime.date(2020, 2, 5), datetime.date(2020, 2, 6),
       datetime.date(2020, 2, 7), datetime.date(2020, 2, 10),
       datetime.date(2020, 2, 11), datetime.date(2020, 2, 12),
       datetime.date(2020, 2, 13), datetime.date(2020, 2, 14),
   

二つの同一判定

In [17]:
np.setdiff1d(workdays1,workdays2)

array([datetime.date(2020, 7, 23), datetime.date(2020, 7, 24),
       datetime.date(2020, 8, 10)], dtype=object)

実は，上の3日は2020年のみの国民の祝日である．これは`jpholiday.between`と`jpholiday.is_holiday`とのバージョン差であると考えられる．`between`の方が正しい．

### 非営業日を取得

In [18]:
def get_not_workdays_jp(start_datetime, end_datetime, to_date=True, end_include=False):
    """
    非営業日を取得(土日＋祝日)
    
    start_datetime: datetime.datetime
        開始時刻のjstのタイムゾーンが指定されたawareなdatetimeか，naiveなdatetime(ローカルなタイムゾーンとして日本を仮定)
    end_datetime: datetime.datetime
        囚虜時刻のjstのタイムゾーンが指定されたawareなdatetimeか，naiveなdatetime(ローカルなタイムゾーンとして日本を仮定)
    to_date: bool
        出力をdatetime.datetimeにするかdatetime.dateにするか
    end_include: bool
        最終日も含めて出力するか
    """
    start_datetime, end_datetime = check_jst_datetimes_to_naive(start_datetime, end_datetime)
    
    def get_not_workdays_gen_jp(start_datetime, end_datetime):
        start_date = start_datetime.date()
        end_date = end_datetime.date()
        if end_include:
            time_delta_days = (end_date - start_date).days + 1  # 最終日も含める
        else:
            time_delta_days = (end_date - start_date).days  # 最終日は含めない

        for i in range(time_delta_days):
            out_datetime = start_datetime + datetime.timedelta(days=i)
            if out_datetime.weekday() in (5,6) or jpholiday.is_holiday(out_datetime):  # 土日か祝日を取得
                if to_date:
                    out_datetime = out_datetime.date()
                yield out_datetime
        
    # workdayのリストを取得
    not_workday_array = np.array([not_workday for not_workday in get_not_workdays_gen_jp(start_datetime, end_datetime)])
    return not_workday_array

In [19]:
#start_datetime = datetime.datetime(2019, 1, 1, 0, 0, 0)
#end_datetime = datetime.datetime(2019, 12, 31, 0, 0, 0)

jst_timezone = timezone("Asia/Tokyo")
start_datetime = jst_timezone.localize(datetime.datetime(2020, 1, 1, 0, 0, 0))
end_datetime = jst_timezone.localize(datetime.datetime(2020, 12, 31, 0, 0, 0))

not_workdays1 = get_not_workdays_jp(start_datetime, end_datetime, to_date=False)

In [20]:
not_workdays1

array([datetime.datetime(2020, 1, 1, 0, 0),
       datetime.datetime(2020, 1, 4, 0, 0),
       datetime.datetime(2020, 1, 5, 0, 0),
       datetime.datetime(2020, 1, 11, 0, 0),
       datetime.datetime(2020, 1, 12, 0, 0),
       datetime.datetime(2020, 1, 13, 0, 0),
       datetime.datetime(2020, 1, 18, 0, 0),
       datetime.datetime(2020, 1, 19, 0, 0),
       datetime.datetime(2020, 1, 25, 0, 0),
       datetime.datetime(2020, 1, 26, 0, 0),
       datetime.datetime(2020, 2, 1, 0, 0),
       datetime.datetime(2020, 2, 2, 0, 0),
       datetime.datetime(2020, 2, 8, 0, 0),
       datetime.datetime(2020, 2, 9, 0, 0),
       datetime.datetime(2020, 2, 11, 0, 0),
       datetime.datetime(2020, 2, 15, 0, 0),
       datetime.datetime(2020, 2, 16, 0, 0),
       datetime.datetime(2020, 2, 22, 0, 0),
       datetime.datetime(2020, 2, 23, 0, 0),
       datetime.datetime(2020, 2, 24, 0, 0),
       datetime.datetime(2020, 2, 29, 0, 0),
       datetime.datetime(2020, 3, 1, 0, 0),
       datetime.da

同様に，すでに取得したholidayと照らし合わせる．

In [21]:
def get_not_workdays_jp_v2(start_datetime, end_datetime, to_date=True, end_include=False):
    """
    非営業日を取得(土日or祝日)
    
    start_datetime: datetime.datetime
        開始時刻のjstのタイムゾーンが指定されたawareなdatetimeか，naiveなdatetime(ローカルなタイムゾーンとして日本を仮定)
    end_datetime: datetime.datetime
        囚虜時刻のjstのタイムゾーンが指定されたawareなdatetimeか，naiveなdatetime(ローカルなタイムゾーンとして日本を仮定)
    to_date: bool
        出力をdatetime.datetimeにするかdatetime.dateにするか
    end_include: bool
        最終日も含めて出力するか
    """
    start_datetime, end_datetime = check_jst_datetimes_to_naive(start_datetime, end_datetime)
    
    # 期間中のholidayを取得
    holidays_in_span_datetimeindex = holidays_datetimeindex[(start_datetime<=holidays_datetimeindex)&(holidays_datetimeindex<end_datetime)].copy()
    holidays_in_span_array = holidays_in_span_datetimeindex.values

    # 期間中のdatetimeのarrayを取得
    if end_include:
        days_siries = pd.date_range(start=start_datetime, end=end_datetime, freq="D")  # 最終日も含める
    else:
        days_siries = pd.date_range(start=start_datetime, end=end_datetime-datetime.timedelta(days=1), freq="D")  # 最終日は含めない
    
    days_datetimeindex = pd.DatetimeIndex(days_siries)
    
    # 休日に含まれないもの，さらに土日に含まれないもののboolインデックスを取得
    holiday_bool_array = days_datetimeindex.isin(holidays_in_span_array)  # 休日
    sun_or_satur_bool_array = (days_datetimeindex.weekday==5) | (days_datetimeindex.weekday==6)  # 土曜日or日曜日
    
    not_workdays_bool_array = holiday_bool_array| sun_or_satur_bool_array  # 休日あるいは土日
    
    not_workdays_datetimeindex = days_datetimeindex[not_workdays_bool_array].copy()
    not_workdays_array = not_workdays_datetimeindex.to_pydatetime()
    if to_date:
        not_workdays_array = convert_datetime2date_ufunc(not_workdays_array)
        
    return not_workdays_array

In [22]:
#start_datetime = datetime.datetime(2019, 1, 1, 0, 0, 0)
#end_datetime = datetime.datetime(2019, 12, 31, 0, 0, 0)

jst_timezone = timezone("Asia/Tokyo")
start_datetime = jst_timezone.localize(datetime.datetime(2020, 1, 1, 0, 0, 0))
end_datetime = jst_timezone.localize(datetime.datetime(2020, 12, 31, 0, 0, 0))

not_workdays2 = get_not_workdays_jp_v2(start_datetime, end_datetime, to_date=False)

In [23]:
not_workdays2

array([datetime.datetime(2020, 1, 1, 0, 0),
       datetime.datetime(2020, 1, 4, 0, 0),
       datetime.datetime(2020, 1, 5, 0, 0),
       datetime.datetime(2020, 1, 11, 0, 0),
       datetime.datetime(2020, 1, 12, 0, 0),
       datetime.datetime(2020, 1, 13, 0, 0),
       datetime.datetime(2020, 1, 18, 0, 0),
       datetime.datetime(2020, 1, 19, 0, 0),
       datetime.datetime(2020, 1, 25, 0, 0),
       datetime.datetime(2020, 1, 26, 0, 0),
       datetime.datetime(2020, 2, 1, 0, 0),
       datetime.datetime(2020, 2, 2, 0, 0),
       datetime.datetime(2020, 2, 8, 0, 0),
       datetime.datetime(2020, 2, 9, 0, 0),
       datetime.datetime(2020, 2, 11, 0, 0),
       datetime.datetime(2020, 2, 15, 0, 0),
       datetime.datetime(2020, 2, 16, 0, 0),
       datetime.datetime(2020, 2, 22, 0, 0),
       datetime.datetime(2020, 2, 23, 0, 0),
       datetime.datetime(2020, 2, 24, 0, 0),
       datetime.datetime(2020, 2, 29, 0, 0),
       datetime.datetime(2020, 3, 1, 0, 0),
       datetime.da

## データフレームに関しての処理

### データの用意 

In [29]:
from get_stock_price import StockDatabase
from pathlib import Path

db_path = Path("db/stock_db") / Path("stock.db")
stock_db = StockDatabase(db_path)

In [30]:
jst_timezone = timezone("Asia/Tokyo")
start_datetime = jst_timezone.localize(datetime.datetime(2020,11,5,9,0,0))
end_datetime = jst_timezone.localize(datetime.datetime(2020,12,1,15,0,0))

stock_df = stock_db.search_span(stock_names=["6502"], 
                                start_datetime=start_datetime,
                                end_datetime=end_datetime,
                                freq_str="T",
                                to_tokyo=True,
                               )
stock_df

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-05 09:00:00+09:00,2631.0,2641.0,2630.0,2639.0,55000.0
2020-11-05 09:01:00+09:00,2639.0,2646.0,2639.0,2644.0,5900.0
2020-11-05 09:02:00+09:00,2645.0,2648.0,2645.0,2646.0,4100.0
2020-11-05 09:03:00+09:00,2646.0,2648.0,2645.0,2645.0,4500.0
2020-11-05 09:04:00+09:00,2645.0,2646.0,2643.0,2643.0,2400.0
...,...,...,...,...,...
2020-12-01 14:55:00+09:00,2950.0,2952.0,2949.0,2951.0,25100.0
2020-12-01 14:56:00+09:00,2950.0,2951.0,2948.0,2949.0,16400.0
2020-12-01 14:57:00+09:00,2950.0,2955.0,2950.0,2954.0,25800.0


### 営業日データを取得

高速化のため，既存のholidayと照らし合わせる方法でとる

In [114]:
def extract_workdays_jp(df, return_as="df"):
    """
    データフレームから，営業日のデータのものを抽出．出力データ形式をreturn_asで指定する．
    df: pd.DataFrame(インデックスとしてpd.DatetimeIndex)
        入力データ
    return_as: str
        出力データの形式
        - "df": 抽出した新しいpd.DataFrameを返す
        - "index": 引数としたdfの対応するインデックスを返す
        - "bool": 引数としたdfに対応するboolインデックスを返す
    """
    
    # 返り値の形式の指定
    return_as_set = {"df", "index", "bool"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))
    
    # すでにtimestampでソートさてている前提
    start_datetime = df.index[0].to_pydatetime()
    end_datetime = df.index[-1].to_pydatetime()
    
    start_datetime, end_datetime = check_jst_datetimes_to_naive(start_datetime, end_datetime)
    
    # 期間内のholidayを取得
    holidays_in_span_datetimeindex = holidays_datetimeindex[(start_datetime<=holidays_datetimeindex)&(holidays_datetimeindex<end_datetime)].copy()
    holidays_in_span_array = holidays_in_span_datetimeindex.values
    
    # 休日に含まれないもの，さらに土日に含まれないもののboolインデックスを取得
    holiday_bool_array = np.in1d(df.index.date, holidays_in_span_array)  # 休日
    sun_or_satur_bool_array = (df.index.weekday==5) | (df.index.weekday==6)  # 土曜日or日曜日
    
    workdays_bool_array = (~holiday_bool_array)&(~sun_or_satur_bool_array)  # 休日でなく土日でない
    if return_as=="bool":  # boolで返す場合
        return workdays_bool_array
    
    workdays_df_indice = df.index[workdays_bool_array]
    if return_as=="index":  # indexで返す場合
        return workdays_df_indice
    
    out_df = df.loc[workdays_df_indice,:].copy()
    return out_df

In [115]:
extracted_stock_df = extract_workdays_jp(stock_df, return_as="df")
extracted_stock_df

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-05 09:00:00+09:00,2631.0,2641.0,2630.0,2639.0,55000.0
2020-11-05 09:01:00+09:00,2639.0,2646.0,2639.0,2644.0,5900.0
2020-11-05 09:02:00+09:00,2645.0,2648.0,2645.0,2646.0,4100.0
2020-11-05 09:03:00+09:00,2646.0,2648.0,2645.0,2645.0,4500.0
2020-11-05 09:04:00+09:00,2645.0,2646.0,2643.0,2643.0,2400.0
...,...,...,...,...,...
2020-12-01 14:55:00+09:00,2950.0,2952.0,2949.0,2951.0,25100.0
2020-12-01 14:56:00+09:00,2950.0,2951.0,2948.0,2949.0,16400.0
2020-12-01 14:57:00+09:00,2950.0,2955.0,2950.0,2954.0,25800.0


### 日中データを取得

おそらく使わない，以下の営業日＋日中内で使う．

In [112]:
def extract_intraday_jp(df, return_as="df"):
    """
    データフレームから，日中(9時から11時半，12時半から15時)のデータのものを抽出．出力データ形式をreturn_asで指定する．
    df: pd.DataFrame(インデックスとしてpd.DatetimeIndex)
        入力データ
    return_as: str
        出力データの形式
        - "df": 抽出した新しいpd.DataFrameを返す
        - "index": 引数としたdfの対応するインデックスを返す
        - "bool": 引数としたdfに対応するboolインデックスを返す
    """
    
    # 返り値の形式の指定
    return_as_set = {"df", "index", "bool"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))    
  
    bool_array = np.array([False]*len(df))
    
    # 午前をTrueに
    am_indice = df.index.indexer_between_time(start_time=datetime.time(9,0), end_time=datetime.time(11,30), include_end=False)
    bool_array[am_indice] = True
    
    # 午後をTrueに
    pm_indice = df.index.indexer_between_time(start_time=datetime.time(12,30), end_time=datetime.time(15,0), include_end=False)
    bool_array[pm_indice] = True
    
    if return_as=="bool":
        return bool_array

    intraday_indice = df.index[bool_array]
    if return_as=="index":
        return intraday_indice
    
    out_df = df.loc[intraday_indice,:].copy()
    return out_df

In [113]:
extracted_df = extract_intraday_jp(stock_df, return_as="df")
extracted_df

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-05 09:00:00+09:00,2631.0,2641.0,2630.0,2639.0,55000.0
2020-11-05 09:01:00+09:00,2639.0,2646.0,2639.0,2644.0,5900.0
2020-11-05 09:02:00+09:00,2645.0,2648.0,2645.0,2646.0,4100.0
2020-11-05 09:03:00+09:00,2646.0,2648.0,2645.0,2645.0,4500.0
2020-11-05 09:04:00+09:00,2645.0,2646.0,2643.0,2643.0,2400.0
...,...,...,...,...,...
2020-12-01 14:55:00+09:00,2950.0,2952.0,2949.0,2951.0,25100.0
2020-12-01 14:56:00+09:00,2950.0,2951.0,2948.0,2949.0,16400.0
2020-12-01 14:57:00+09:00,2950.0,2955.0,2950.0,2954.0,25800.0


### 営業日＋日中データを取得 

In [116]:
def extract_workdays_intraday_jp(df, return_as="df"):
    """
    データフレームから，営業日+日中(9時から11時半，12時半から15時)のデータのものを抽出．出力データ形式をreturn_asで指定する．
    df: pd.DataFrame(インデックスとしてpd.DatetimeIndex)
        入力データ
    return_as: str
        出力データの形式
        - "df": 抽出した新しいpd.DataFrameを返す
        - "index": 引数としたdfの対応するインデックスを返す
        - "bool": 引数としたdfに対応するboolインデックスを返す
    """
    
    # 返り値の形式の指定
    return_as_set = {"df", "index", "bool"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))    
       
    
    workday_bool_array = extract_workdays_jp(df, return_as="bool")
    intraday_bool_array = extract_intraday_jp(df, return_as="bool")
    
    workday_intraday_bool_array = workday_bool_array & intraday_bool_array
    
    if return_as=="bool":
        return workday_intraday_bool_array
    
    workday_intraday_indice = df.index[workday_intraday_bool_array]
    
    if return_as=="index":
        return workday_intraday_indice
    
    out_df = df.loc[workday_intraday_indice,:].copy()
    return out_df

In [120]:
extracted_df = extract_workdays_intraday_jp(stock_df, return_as="df")
extracted_df

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-05 09:00:00+09:00,2631.0,2641.0,2630.0,2639.0,55000.0
2020-11-05 09:01:00+09:00,2639.0,2646.0,2639.0,2644.0,5900.0
2020-11-05 09:02:00+09:00,2645.0,2648.0,2645.0,2646.0,4100.0
2020-11-05 09:03:00+09:00,2646.0,2648.0,2645.0,2645.0,4500.0
2020-11-05 09:04:00+09:00,2645.0,2646.0,2643.0,2643.0,2400.0
...,...,...,...,...,...
2020-12-01 14:55:00+09:00,2950.0,2952.0,2949.0,2951.0,25100.0
2020-12-01 14:56:00+09:00,2950.0,2951.0,2948.0,2949.0,16400.0
2020-12-01 14:57:00+09:00,2950.0,2955.0,2950.0,2954.0,25800.0
